# 感知器（Perceptron）

## Definition

$$y = f(x) = sign(\omega \cdot x + b), $$

$$sign(x) = \left\{ \begin{array}{rcl} +1,& x\geq0 \\ -1, & x<0 \end{array} \right.$$

where $\omega \in \mathcal{R^n}, \ b \in \mathcal{R}, x \in \mathcal{X}, y \in \mathcal{Y}$

$\mathcal{X} \subseteq \mathcal{R^n}$ is the input space 

$\mathcal{Y} = \{+1, -1\}$ is the output space

### Train Set
$$T = \{ (\textbf{x}_1, \textbf{y}_1), (\textbf{x}_2, \textbf{y}_2), \cdots, (\textbf{x}_n, \textbf{y}_n) \},$$ 

where $\textbf x_i \in \mathcal{X}, \ y_i \in \mathcal{Y}, \ i = 1, 2, \cdots, N$

### Form

$$y = f(x) = sign(\omega \cdot x + b) = sign(\omega^T \textbf{x}), $$

where $\omega = [b, \omega_1, \omega_2, \cdots, \omega_n]^T, \textbf{x}_i = [+1, x_{i1}, x_{i2}, \cdots, x_{in}]^T$


## Learning Algorithm

$$\omega(n+1) = \left\{ \begin{array}{rcl} \omega(n), & \hat{y}(n) = y(n) \\ \omega(n) + \eta(n)\textbf{x}(n), & \hat{y}(n) \neq y(n) & y(n)=1 \\ \omega(n) - \eta(n)\textbf{x}(n), & \hat{y}(n) \neq y(n) & y(n) =-1\end{array}\right.,$$

obtain:

$$\omega(n+1) = \omega(n) + \eta y(n)\textbf{x}(n)$$

where

 $\eta(n)$ is the learning rate parameter

 $\hat{y}(n) = f(\textbf{x}(n)) = sign(\omega \cdot \textbf{x}(n) + b)$ is the output of the $n$ th member of the training set

 $y(n)$ is the label of the $n$ th member of the training set

 $\textbf{x}(n)$ is the $n$ th member of the training set

 ## Convergence

 ### Novikoff